In [1]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint

import pytorch_lightning as plpl

import torch

from helper import load_dataset
from model import Data, SoftMaxLit, TransformerModel, get_dataloaders, LogisticRegression

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
device = torch.cuda.current_device()

In [5]:
device

0

In [6]:
BATCH_SIZE = 4
NUM_EPOCH = 4
DEV = True

# Because DEV
df = load_dataset('../dataset/training.json', test=True)[:100]

model_names = ['albert', 'electra', 'roberta', 'xlnet']

In [7]:
# for cur_model_name in model_names:
#     TransformerModel(cur_model_name).dataset(df, DEV, save=True)

In [8]:
# for cur_model_name in model_names:
#     cur_model_name
#     cur_dataset_x = torch.load(f'pretrained--dev=True--model={cur_model_name}.pt')
#     cur_data = Data(df, x=cur_dataset_x)
#     cur_dataloaders = get_dataloaders(cur_data, BATCH_SIZE)
#     cur_model = SoftMaxLit(TransformerModel.MODELS[cur_model_name]['dim'], 2)
#     checkpoint_callback = ModelCheckpoint(
#         save_top_k=1,
#         monitor='val_loss',
#         mode='min',
#         filename=f'model={cur_model_name}--dev={DEV}' + '--{epoch}-{step}--{val_loss:.2f}'
#     )

#     trainer = pl.Trainer(callbacks=[checkpoint_callback], max_epochs=NUM_EPOCH)
#     trainer.fit(model=cur_model, train_dataloaders=cur_dataloaders['train'], val_dataloaders=cur_dataloaders['val'])
#     trainer.test(cur_model, dataloaders=cur_dataloaders['test'])

In [9]:
pretrained_datasets_x = [
    "pretrained--dev=True--model=albert.pt",
    "pretrained--dev=True--model=electra.pt",
    "pretrained--dev=True--model=roberta.pt",
    "pretrained--dev=True--model=xlnet.pt"
]

checkpoints = [
    'lightning_logs/version_0/checkpoints/model=albert--dev=True--epoch=2-step=60--val_loss=0.39.ckpt',
    'lightning_logs/version_1/checkpoints/model=electra--dev=True--epoch=2-step=60--val_loss=0.53.ckpt',
    'lightning_logs/version_4/checkpoints/model=roberta--dev=True--epoch=3-step=80--val_loss=0.63.ckpt',
    'lightning_logs/version_5/checkpoints/model=xlnet--dev=True--epoch=3-step=80--val_loss=0.62.ckpt'
]

model_y_arr = []
for model_name, pretrained_dataset_x, ckpt in zip(model_names, pretrained_datasets_x, checkpoints):
    n_inputs = TransformerModel.MODELS[model_name]['dim']
    model = SoftMaxLit(n_inputs, 2).load_from_checkpoint(n_inputs=n_inputs, n_outputs=2, checkpoint_path=ckpt)
    x = torch.load(pretrained_dataset_x).to(device)
    y_hat = model(x)
    
    # Free up memory
    del x
    torch.cuda.empty_cache()
#     y_first = y_hat[:,0]
    y_first = y_hat
#     y_first = y_first.reshape((y_first.shape[0], 1))

    model_y_arr.append(y_first)

lr_dataset_x = torch.cat(model_y_arr, dim=1).detach()

In [10]:
NUM_EPOCH = 100

lr_dataset = Data(df, x=lr_dataset_x, one_hot=True)
lr_dataloaders = get_dataloaders(lr_dataset, BATCH_SIZE)

# lr_model = LogisticRegression(input_dim=lr_dataset_x.shape[1], num_classes=2)
lr_model = SoftMaxLit(lr_dataset_x.shape[1], 2)

checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    monitor='val_loss',
    mode='min',
    filename=f'model=lr--dev={DEV}'
)

trainer = pl.Trainer(callbacks = [checkpoint_callback], max_epochs=NUM_EPOCH) # callbacks=[checkpoint_callback]
trainer.fit(model=lr_model, train_dataloaders=lr_dataloaders['train'], val_dataloaders=lr_dataloaders['val'])
trainer.test(lr_model, dataloaders=lr_dataloaders['test'])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2023-09-16 19:49:57.147480: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-16 19:49:57.178393: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To 

Sanity Checking: 0it [00:00, ?it/s]

/home/duke/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/home/duke/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/duke/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machi

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/duke/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/home/duke/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │            1.0            │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 1.0}]

In [11]:
x = lr_dataset.x.to(device)
lr_model.to(device)
lr_model(x)

tensor([[0.2568, 0.7432],
        [0.2257, 0.7743],
        [0.0480, 0.9520],
        [0.0417, 0.9583],
        [0.9722, 0.0278],
        [0.9778, 0.0222],
        [0.0268, 0.9732],
        [0.6503, 0.3497],
        [0.0266, 0.9734],
        [0.0287, 0.9713],
        [0.2229, 0.7771],
        [0.1015, 0.8985],
        [0.0569, 0.9431],
        [0.9486, 0.0514],
        [0.0411, 0.9589],
        [0.0724, 0.9276],
        [0.9454, 0.0546],
        [0.8980, 0.1020],
        [0.0364, 0.9636],
        [0.0279, 0.9721],
        [0.9049, 0.0951],
        [0.0800, 0.9200],
        [0.0608, 0.9392],
        [0.9676, 0.0324],
        [0.9567, 0.0433],
        [0.9600, 0.0400],
        [0.1731, 0.8269],
        [0.0225, 0.9775],
        [0.9627, 0.0373],
        [0.0512, 0.9488],
        [0.9067, 0.0933],
        [0.8581, 0.1419],
        [0.9790, 0.0210],
        [0.9796, 0.0204],
        [0.0613, 0.9387],
        [0.0678, 0.9322],
        [0.0362, 0.9638],
        [0.0268, 0.9732],
        [0.0

In [12]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/